In [1]:
import pandas as pd
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn import metrics
pd.options.display.max_columns = 1200
pd.options.display.max_rows = 1200

# Read in and Prep Data

In [2]:
# df = pd.read_csv('data/autoperf_filtered_1000000ch', header=0).sample(frac=1)
df = pd.read_csv('data/composed/mira_top_20.csv', header=0).sample(frac=1)

# li = []
# for filename in glob.glob("/home/luckierdodge/repos/jupyter-notebooks/data/autoperf_complete/autoperf-201*.csv"):
#     frame = pd.read_csv(filename, header=0)
#     li.append(frame)
# df = pd.concat(li, axis=0, ignore_index=True)

# df.dropna(inplace=True);

/home/luckierdodge/anaconda3/envs/autoperf_tf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (876) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# counts = df.zero_execName.value_counts().to_dict()
# df_filtered = pd.DataFrame()
# for name in counts:
#     if counts[name] >= 500:
#         df_filtered = df_filtered.append(df[df["zero_execName"] == name])
# df = df_filtered.copy()

In [4]:
df.zero_elapsedTime.describe()

count    94408.000000
mean      6146.709119
std      10552.441888
min          0.000425
25%        288.420416
50%       1283.770563
75%      10124.530355
max      85970.668079
Name: zero_elapsedTime, dtype: float64

In [5]:
train_size = .8
train = df.head(int(df.size * train_size))
test = df.tail(int(df.size * (1-train_size)))

In [6]:
features = ["zero_execName", "zero_userName"]
# targets = ["zero_elapsedTime"]
targets = ["zero_numRanks"]

X_train = train[features]
X_test = test[features]
Y_train = train[targets]
Y_test = train[targets]

# Linear Regression using SciKit Learn (One-Hot Encoding)

In [7]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=True)
enc.fit(X_train.values)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [8]:
X_train_one_hot = enc.transform(X_train.values)
X_test_one_hot = enc.transform(X_test.values)

In [9]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)
model.fit(X_train_one_hot, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [10]:
model.score(X_test_one_hot, Y_test)

0.3963747721976786

In [11]:
import math
Y_predict = model.predict(X_test_one_hot)
regression_model_mse = metrics.mean_squared_error(Y_predict, Y_test)
math.sqrt(regression_model_mse)

22820.316424805675

In [12]:
print(math.sqrt(regression_model_mse) / (max(Y_test.values) - min(Y_test.values)))

[0.02176317]


# Keras Analysis

In [13]:
import tensorflow as tf

In [14]:
train_examples = X_train_one_hot
train_targets = Y_train
test_examples = X_test_one_hot
test_targets = Y_test

In [15]:
train_targets.shape

(94408, 1)

In [16]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=(train_examples.shape[1],)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(64, activation='relu',))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(1))

W0716 14:13:33.271563 139817853818688 deprecation.py:506] From /home/luckierdodge/anaconda3/envs/autoperf_tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(optimizer=optimizer,
             loss='mean_squared_error',
             metrics=['mean_absolute_error', 'mean_squared_error'],
             )

In [18]:
model.fit(train_examples, train_targets, epochs=10)

Epoch 1/10
94408/94408 [==============================] - 9s 97us/sample - loss: 887164720.7895 - mean_absolute_error: 5439.9517 - mean_squared_error: 887165248.0000
Epoch 2/10
94408/94408 [==============================] - 8s 88us/sample - loss: 862777567.8847 - mean_absolute_error: 5071.3638 - mean_squared_error: 862777088.0000
Epoch 3/10
94408/94408 [==============================] - 8s 88us/sample - loss: 823365893.4993 - mean_absolute_error: 4976.5176 - mean_squared_error: 823366208.0000
Epoch 4/10
94408/94408 [==============================] - 8s 87us/sample - loss: 780231337.5039 - mean_absolute_error: 5232.6919 - mean_squared_error: 780230848.0000
Epoch 5/10
94408/94408 [==============================] - 8s 88us/sample - loss: 738988956.2256 - mean_absolute_error: 5521.5889 - mean_squared_error: 738988992.0000
Epoch 6/10
94408/94408 [==============================] - 8s 88us/sample - loss: 706613726.4364 - mean_absolute_error: 5728.8140 - mean_squared_error: 706613376.0000
Epoc

In [19]:
test_loss, test_abs_err, test_sqr_err = model.evaluate(test_examples, test_targets)
# print('Test accuracy:', test_acc)

94408/94408 [==============================] - 20s 207us/sample - loss: 571386461.8457 - mean_absolute_error: 4039.2441 - mean_squared_error: 571386112.0000


In [20]:
print(test_abs_err / (max(test_targets.values) - min(test_targets.values)))

[0.00385213]
